In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from importlib import reload

# DBSCAN import
from sklearn.cluster import DBSCAN
# Scaler import
from sklearn.preprocessing import MinMaxScaler

# import modules
import src.wrangle as wr

# define the default font sizes
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [3]:
df = wr.get_logs()

In [4]:
df.head()

,date_time,date,page,id,cohort,ip,cohort_name,topic,lesson,Topic,Lesson,full_lesson_name,field
0,2018-01-26 09:55:03,2018-01-26,index,1,8,97.105.19.61,Hampton,index,index,index,index,index,WebDev
1,2018-01-26 09:56:02,2018-01-26,java-ii,1,8,97.105.19.61,Hampton,java-ii,java-ii,java-ii,java-ii,java-ii,WebDev
2,2018-01-26 09:56:05,2018-01-26,java-ii/object-oriented-programming,1,8,97.105.19.61,Hampton,java-ii,object-oriented-programming,java-ii,object-oriented-programming,java-ii object-oriented-programming,WebDev
3,2018-01-26 09:56:06,2018-01-26,slides/object_oriented_programming,1,8,97.105.19.61,Hampton,slides,object_oriented_programming,slides,object_oriented_programming,slides object_oriented_programming,WebDev
4,2018-01-26 09:56:24,2018-01-26,javascript-i/conditionals,2,22,97.105.19.61,Teddy,javascript-i,conditionals,javascript-i,conditionals,javascript-i conditionals,WebDev
